# **C-HACK Tutorial 5: Data Regression Analysis**

**Instructor**: Stephanie Valleau <br>
**Contact**: valleau@uw.edu

**Acknowlegments:** Stephanie would like acknowledge the entire C-Hack hackmaster team for review and comments on the tutorial. In particular, this tutorial was reviewed by:

* Kenny Andre
* Jordy Jackson



In this tutorial we will look at fitting data to a curve using **regression**. We will also look at using regression to make **predictions** for new data points by dividing our data into a training and a testing set. Finally we will examine how much error we make in our fit and then in our predictions by computing the mean squared error, at the variance and bias.



---







### Load libraries which will be needed in this Notebook



In [1]:
# Pandas library for the pandas dataframes
import pandas as pd    

# Import Scikit-Learn library for the regression models
import sklearn         
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Import numpy 
import numpy as np

# Import plotting libraries
import seaborn as sns
import matplotlib 
from matplotlib import pyplot as plt

# Set larger fontsize for all plots
matplotlib.rcParams.update({'font.size': 20})

# Command to automatically reload modules before executing cells
# not needed here but might be if you are writing your own library 
%load_ext autoreload
%autoreload 2


### Load our google drive folder & find path of files

When you execute the cell below you will be asked to give authorization to colab to access your drive. 

In [2]:
# Sync your google drive folder
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


Follow the instructions shown in *Evan Komp's* [video](https://youtu.be/5rZn-aVNR0A) to locate the file called "running_cheetah.csv" and save its path in a variable

In [6]:
your_path_to_cheetah_file="/content/drive/MyDrive/C-HACK 2021 EVENT/Tutorials/Tutorial 5/running_cheetah.csv"

## 5.1 What is regression? 

It is the process of finding a relationship between **_dependent_** and **_independent_** variables to find trends in data. This abstract definition means that you have one variable (the dependent variable) which depends on one or more variables (the independent variables). One of the reasons for which we want to regress data is to understand whether there is a trend between two variables. For instance, let's say we had data points showing how tired we are respect to the amount of coffee we drink. In the case of linear regression, the question would be - is the data quantity of how tired I am linearly proportional to the amount of coffee I drink, i.e. can we use a line to represent the relationship between these two variables?




## 5.2  Linear regression + a Cheetah: fitting with scikit-learn

### 5.2.1 Loading the dataset

Consider a Cheetah on the run (worth watching): https://www.youtube.com/watch?v=8-9oFxYFODE
<br>


![Image of Cheetah](https://drive.google.com/uc?export=view&id=1lpZinFnWbbreZxx2BxpldWPVBlAyUL89)

Assume you have a set of data with values (time, position, speed, energy) for one Cheetah's motion. The dataset is in csv format, lets load it by using the <code> Pandas </code> library - the name of the file is <code> running_cheetah.csv </code>



```
# Load the dataset using the read_csv() pandas function - we have to indicate that
# the index of each row is in the first column
cheetah_df=pd.read_csv("running_cheetah.csv",index_col=0)
```



In [7]:
# Load the dataset using the read_csv() pandas function - we have to indicate that
# the index of each row is in the first column
cheetah_df=pd.read_csv(your_path_to_cheetah_file,index_col=0)

#### Exercise 1
What does the data look like? Remember how to visualize data in a pandas dataframe (Tutorial 2) 


### 5.2.2 Visualizing the data set

We can create a scatter plot of positions of the Cheetah as a function of time using the <code> plt.scatter() </code> function (as discussed in Tutorial 4)

#### Exercise 2 
Create a scatter plot of positions of the Cheetah respect to time


### 5.2.3 Linear regression - fitting data to a line

It looks like position keeps increasing in time following a line, maybe something like

$$y(t)= m\cdot t + b  \;\;\;\;\;\;\;\;\sf{eq. 1}$$ 

with $y=\sf position$, $t=\sf time$, $m$ the slope and $b$ the intercept. 

To solve the problem, we need to find the values of $b$ and $m$ in equation 1 to best fit the data. This is called **linear regression**.

In linear regression our goal is to minimize the error between computed values of positions $y^{\sf calc}(t_i)\equiv y^{\sf calc}_i$ and known values $y^{\sf exact}(t_i)\equiv y^{\sf exact}_i$, i.e. find $b$ and $m$ which lead to lowest value of

$$\epsilon (m,b) =\sum_{i=1}^{N_{\sf time points}}\left(y^{\sf exact}_i - y^{\sf calc}_i\right)^2 = \sum_{i=1}^{N_{\sf time points}}\left(y^{\sf exact}_i - m\cdot t_i - b \right)^2\;\;\;\;\;\;\;\;\;\;\;\sf{eq. 2}$$

To find out more see e.g. https://en.wikipedia.org/wiki/Simple_linear_regression

#### Question 1
Do we always want *m* and *b* to be large positive numbers so as to minimize eq. 2? Respond to the **polleverywhere** quiz - https://pollev.com/weloveds 

### 5.2.4 The scikit-learn library

Luckily there is a Python library called [scikit-learn](https://scikit-learn.org/stable/) which contains many functions related to regression including [linear regression](https://scikit-learn.org/stable/modules/linear_model.html). 

The function we will use is called <code> LinearRegression() </code>. 

### 5.2.5 Fitting the Cheetah position in time data to a line

```
# Create linear regression object
regr = linear_model.LinearRegression()

# Use model to fit to the data, the X values are times and the Y values are positions of the Cheetah
# Note that we need to reshape the vectors to be of the shape X - (n_samples, n_features) and Y (n_samples, n_targets)
X = cheetah_df['time [hr]'].values.reshape(-1, 1)
Y = cheetah_df['position [miles]'].values.reshape(-1, 1)
```

In [ ]:
# Create linear regression object
regr = linear_model.LinearRegression()

# Use model to fit to the data, the X values are times and the Y values are positions of the Cheetah
# Note that we need to reshape the vectors to be of the shape X - (n_samples, n_features) and Y (n_samples, n_targets)
X = cheetah_df['time [hr]'].values.reshape(-1, 1)
Y = cheetah_df['position [miles]'].values.reshape(-1, 1)


```
print(cheetah_df['time [hr]'].values.shape, cheetah_df['position [miles]'].values.shape)
print(X.shape, Y.shape)
```

In [ ]:
print(cheetah_df['time [hr]'].values.shape, cheetah_df['position [miles]'].values.shape)
print(X.shape, Y.shape)

```
# Fit to the data
regr.fit(X, Y)

# Extract the values of interest
m = regr.coef_[0][0]
b = regr.intercept_[0]

# Print the slope m and intercept b
print('Scikit learn - Slope: ', m , 'Intercept: ', b )
```

In [ ]:
# Fit to the data
regr.fit(X, Y)

# Extract the values of interest
m = regr.coef_[0][0]
b = regr.intercept_[0]

# Print the slope m and intercept b
print('Scikit learn - Slope: ', m , 'Intercept: ', b )

#### Exercise 3
Estimate the values of $y$ by using your fitted parameters. Hint: Use your <code>regr.coef_</code> and <code>regr.intercept_</code> parameters to estimate Y_calc_2 following equation 1


In [ ]:
# Another way to get this is using the regr.predict function
Y_calc = regr.predict(X)

Let's create a graphical visualization of the fitted values respect to the exact values

```
# Lets plot exact positions respect to the time values using a scatter plot
plt.scatter(cheetah_df['time [hr]'], cheetah_df['position [miles]'], s=100, marker='.', color="cornflowerblue")
plt.xlabel('time [hours]')
plt.ylabel('position [miles]')

# Now we compare to our fit by plotting both Y_cal and Y_calc_2 respect to time 
plt.plot(X, Y_calc, color='crimson',linewidth=3, label='linear fit')
plt.plot(X, Y_calc_2,':', color='black',linewidth=3, label='linear fit check')
plt.legend(bbox_to_anchor=(1.8, 1), loc='upper right')
plt.show()
```

In [ ]:
# Lets plot exact positions respect to the time values using a scatter plot
plt.scatter(cheetah_df['time [hr]'], cheetah_df['position [miles]'], s=100, marker='.', color="cornflowerblue")
plt.xlabel('time [hours]')
plt.ylabel('position [miles]')

# Now we compare to our fit by plotting both Y_cal and Y_calc_2 respect to time 
plt.plot(X, Y_calc, color='crimson',linewidth=3, label='linear fit')
plt.plot(X, Y_calc_2,':', color='black',linewidth=3, label='linear fit check')
plt.legend(bbox_to_anchor=(1.8, 1), loc='upper right')
plt.show()

## 5.3 How much error are we making? 

### 5.3.1 Mean Squared Error

The plot in Section 5.2.5 looks good, but numerically what is our error? What is the mean value of $\epsilon$, i.e. the **Mean Squared Error (MSE)**?

$${\sf MSE}=\epsilon_{\sf ave} = \frac{\sum_{i=1}^{N_{\sf times}}\left(y^{\sf exact}_i - m\cdot t_i - b \right)^2}{N_{\sf times}}\;\;\;\;\;\sf eq. 3$$

```
# The mean squared error
print('Mean squared error: %.2f' % mean_squared_error(Y, Y_calc))
```

In [ ]:
# The mean squared error
print('Mean squared error: %.2f' % mean_squared_error(Y, Y_calc))

Another way to measure error is the regression score, $R^2$. $R^2$ is generally defined as the ratio of the total sum of squares $SS_{\sf tot} $ to the residual sum of squares $SS_{\sf res} $:

$$SS_{\sf tot}=\sum_{i=1}^{N} \left(y^{\sf exact}_i-\bar{y}\right)^2\;\;\;\;\; \sf eq. 4$$
$$SS_{\sf res}=\sum_{i=1}^{N} \left(y^{\sf exact}_i - y^{\sf calc}_i\right)^2\;\;\;\;\; \sf eq. 5$$
$$R^2 = 1 - {SS_{\sf res}\over SS_{\sf tot}} \;\;\;\;\;\; \sf eq. 6$$

In eq. 4, $\bar{y}=\sum_i y^{\sf exact}_i/N$ is the average value of y for $N$ points. The best value of $R^2$ is 1 but it can also take a negative value if the error is large.

See all the different regression metrics [here](https://scikit-learn.org/stable/modules/model_evaluation.html).

#### Question 3
Do we need a large value of $SS_{\sf tot}$ to minimize $R^2$ - is this something which we have the power to control? Respond to the **polleverywhere** quiz - https://pollev.com/weloveds 

```
# Print the coefficient of determination - 1 is perfect prediction
print('Coefficient of determination: %.2f' % r2_score(Y, Y_calc))
```

In [ ]:
# Print the coefficient of determination - 1 is perfect prediction
print('Coefficient of determination: %.2f' % r2_score(Y, Y_calc))

### 5.3.2 Predicting with a fitted model - Bias and Variance

We could have used a part of the data to fit our line - **_training_** data - and the rest to validate it - **_test_** data. We will see an example of this in Section 5.4. At that point, the <code> predict() </code> function would have predicted previoulsy unseen positions. In the case when a subset of training data is used to find the parameters of the model we refer to the model as *_trained_* once the optimal values of the parameters have been determined.

Here, we are interested in evaluating the accuracy of the trained model when it predicts data from the test dataset. The error can be divided in to a **_irreducible_** and **_reducible_** part. 

The **_irreducible error_** comes from the fact that data is noisy.



For more information see [here](https://www.coursera.org/lecture/ml-regression/irreducible-error-and-bias-qlMrZ). 

The **_reducible error_** of the model , i.e. the error which comes from the mismatch of the model function $y=m_{\sf opt}x+b_{\sf opt}$ respect to the exact data $\left\{y_i\right\}_{i=1}^N$ is then broken down into **_bias_** and **_variance_**. To understand what these two terms mean we will consider the problem of the Cheetah. 


### 5.3.2.1 Variance

Let's go back to our Cheetah problem and train the model on three different sets of training points

```
# Define three datasets from the original set by selecting a range of values using the np.argwhere() function

# Search for positions in each numpy array where the values are in a specific range of time e.g. time < 35 hr
indices_test = [ind[0] for ind in np.argwhere(X <= 35)]
indices1 = [ind[0] for ind in np.argwhere((X > 35) & (X < 50))]
indices2 = [ind[0] for ind in np.argwhere((X >= 50) & (X < 70))]
indices3 = [ind[0] for ind in np.argwhere(X >= 70)]

# Define the three training dataset and testing datasets by using the indices 
X1_train = X[indices1] 
Y1_train = Y[indices1] 

X2_train = X[indices2]
Y2_train = Y[indices2]

X3_train = X[indices3]
Y3_train = Y[indices3]

X_test = X[indices_test]
Y_test = Y[indices_test]
```

In [ ]:
# Define three datasets from the original set by selecting a range of values using the np.argwhere() function

# Search for positions in each numpy array where the values are in a specific range of time e.g. time < 35 hr
indices_test = [ind[0] for ind in np.argwhere(X <= 35)]
indices1 = [ind[0] for ind in np.argwhere((X > 35) & (X < 50))]
indices2 = [ind[0] for ind in np.argwhere((X >= 50) & (X < 70))]
indices3 = [ind[0] for ind in np.argwhere(X >= 70)]

# Define the three training dataset and testing datasets by using the indices 
X1_train = X[indices1] 
Y1_train = Y[indices1] 

X2_train = X[indices2]
Y2_train = Y[indices2]

X3_train = X[indices3]
Y3_train = Y[indices3]

X_test = X[indices_test]
Y_test = Y[indices_test]


```
# Let's plot the three training sets and the test set
plt.scatter(X1_train, Y1_train, s=100, marker='.', color="royalblue", label="training set 1")
plt.scatter(X2_train, Y2_train, s=100, marker='.', color="indianred", label="training set 2")
plt.scatter(X3_train, Y3_train, s=100, marker='.', color="mediumseagreen", label="training set 3")
plt.scatter(X_test, Y_test, s=100, marker='.', color="grey", label="test set 3")
plt.xlabel('time [hours]')
plt.ylabel('position [miles]')
plt.legend(bbox_to_anchor=(1.8, 1), loc='upper right')
plt.show()
```

In [ ]:
# Let's plot the three training sets and the test set
plt.scatter(X1_train, Y1_train, s=100, marker='.', color="royalblue", label="training set 1")
plt.scatter(X2_train, Y2_train, s=100, marker='.', color="indianred", label="training set 2")
plt.scatter(X3_train, Y3_train, s=100, marker='.', color="mediumseagreen", label="training set 3")
plt.scatter(X_test, Y_test, s=100, marker='.', color="grey", label="test set 3")
plt.xlabel('time [hours]')
plt.ylabel('position [miles]')
plt.legend(bbox_to_anchor=(1.8, 1), loc='upper right')
plt.show()

Now we will proceed just like we did in Section 5.2.5 and carry out linear regression on each of these three training sets

```
regr1 = linear_model.LinearRegression()
regr1.fit(X1_train, Y1_train)

regr2 = linear_model.LinearRegression()
regr2.fit(X2_train, Y2_train)

regr3 = linear_model.LinearRegression()
regr3.fit(X3_train, Y3_train)

m1 = regr1.coef_[0][0]
b1 = regr1.intercept_[0]

m2 = regr2.coef_[0][0]
b2 = regr2.intercept_[0]

m3 = regr3.coef_[0][0]
b3 = regr3.intercept_[0]
```

In [ ]:
regr1 = linear_model.LinearRegression()
regr1.fit(X1_train, Y1_train)

regr2 = linear_model.LinearRegression()
regr2.fit(X2_train, Y2_train)

regr3 = linear_model.LinearRegression()
regr3.fit(X3_train, Y3_train)

m1 = regr1.coef_[0][0]
b1 = regr1.intercept_[0]

m2 = regr2.coef_[0][0]
b2 = regr2.intercept_[0]

m3 = regr3.coef_[0][0]
b3 = regr3.intercept_[0]

```
# Let's see how well they fit/predict the total dataset (note that usually we would only look at the test set here)
Y_calc_1 = regr1.predict(X_test)
Y_calc_2 = regr2.predict(X_test)
Y_calc_3 = regr3.predict(X_test)
```

In [ ]:
# Let's see how well they fit/predict the total dataset (note that usually we would only look at the test set here)
Y_calc_1 = regr1.predict(X_test)
Y_calc_2 = regr2.predict(X_test)
Y_calc_3 = regr3.predict(X_test)

Let's visualize the fitted values using our trained models and compare to the average fit

```
# Lets plot exact positions respect to the time values using a scatter plot
plt.scatter(X_test, Y_test, s=100, marker='o', facecolors='lightgrey', edgecolors='slategrey', label="test data")
plt.xlabel('time [hours]')
plt.ylabel('position [miles]')

# Compute the average calculated position
Y_calc_ave =  ( Y_calc_1 + Y_calc_2 + Y_calc_3 ) / 3

# Now we compare to our fit by plotting both Y_cal and Y_calc_2 respect to time 
plt.plot(X_test, Y_calc_1, color='royalblue',linewidth=3, label='linear fit 1')
plt.plot(X_test, Y_calc_2, color='indianred',linewidth=3, label='linear fit 2')
plt.plot(X_test, Y_calc_3, color='mediumseagreen',linewidth=3, label='linear fit 3')
plt.plot(X_test, Y_calc_ave, color='dimgrey',linewidth=3, label='average fit')
plt.legend(bbox_to_anchor=(1.8, 1), loc='upper right')
plt.show()
```

In [ ]:
# Lets plot exact positions respect to the time values using a scatter plot
plt.scatter(X_test, Y_test, s=100, marker='o', facecolors='lightgrey', edgecolors='slategrey', label="test data")
plt.xlabel('time [hours]')
plt.ylabel('position [miles]')

# Compute the average calculated position
Y_calc_ave =  ( Y_calc_1 + Y_calc_2 + Y_calc_3 ) / 3

# Now we compare to our fit by plotting both Y_cal and Y_calc_2 respect to time 
plt.plot(X_test, Y_calc_1, color='royalblue',linewidth=3, label='linear fit 1')
plt.plot(X_test, Y_calc_2, color='indianred',linewidth=3, label='linear fit 2')
plt.plot(X_test, Y_calc_3, color='mediumseagreen',linewidth=3, label='linear fit 3')
plt.plot(X_test, Y_calc_ave, color='dimgrey',linewidth=3, label='average fit')
plt.legend(bbox_to_anchor=(1.8, 1), loc='upper right')
plt.show()

#### Question 4
Why do you think each linear fit has a different slope and intercept? Respond to the **polleverywhere** quiz - https://pollev.com/weloveds 


In the plot we see that each trained model has a different slope and intercept. The average fit is shown in grey. The difference between what one of the trained model predicts and the average model prediction is what we call the **_variance_**.

The variance is the amount by which the prediction will change if different subsets of the training data sets are used. We generally want to have a low variance as that ensure that our model is not sensitive to small fluctuations in the dataset.

Large variance occurs when the model performs well on the training dataset but does not do well on the test dataset.

If we use more complex models to fit, for example polynomials, that will generally lead to a larger variance.

### 5.3.2.2 Bias

Now, let's imagine that we are given the true universal Cheetah position function, lets call it <code> true_cheetah </code> we define it below

```
def true_cheetah(t):
    return 63.64 * t +  155.03
```

In [ ]:
def true_cheetah(t):
    return 63.64 * t +  155.03

How well are we doing in terms of our prediction error respect to the true values? Are we biased? Let's visualize our results

```
plt.plot(X_test, true_cheetah(X_test), color='mediumseagreen',linewidth=3, label='true cheetah motion')
plt.plot(X_test, Y_calc_ave, color='dimgrey',linewidth=3, label='average fit')
plt.scatter(X_test, Y_test, s=100, marker='o', facecolors='lightgrey', edgecolors='slategrey', label="test data")
plt.xlabel('time [hours]')
plt.ylabel('position [miles]')
plt.legend(bbox_to_anchor=(1.9, 1), loc='upper right')
plt.show()
```

In [ ]:
plt.plot(X_test, true_cheetah(X_test), color='mediumseagreen',linewidth=3, label='true cheetah motion')
plt.plot(X_test, Y_calc_ave, color='dimgrey',linewidth=3, label='average fit')
plt.scatter(X_test, Y_test, s=100, marker='o', facecolors='lightgrey', edgecolors='slategrey', label="test data")
plt.xlabel('time [hours]')
plt.ylabel('position [miles]')
plt.legend(bbox_to_anchor=(1.9, 1), loc='upper right')
plt.show()

We see that our model is biased, indeed our average fit model predicts with a **_bias_** error respect to the true cheetah motion. 

In this case, a more complex function, such as a polynomial might give a better fit and a lower **_bias_**.
<div>

<img src="attachment:cheetah_over_underfitting.png" align=left width="400"/>
</div>



<div>
<img src=https://drive.google.com/uc?export=view&id=1DXLgVKjzBrofSPtxtNcVb9Lf_FbUMcKe width="500">
</div>

We usually want to find a sweet spot which minimizes both bias and variance. As we increase complexity, for instance by using a polynomial fit, our bias goes down however variance increases. Similarly for a very simple model the bias is large while the variance is small. We can see this graphically in the sketch copied below



<div>
<img src=https://drive.google.com/uc?export=view&id=1phnDnjjGJ8twf7h9JKS3TbbN5QlBFj9o width="500">
</div>

### 5.3.3 Beyond a single input feature

The **speed** or velocity of the Cheetah (the dependent variable v) could depend on:
* his/her **energy** (independent variable E) that day
* how well the Cheetah **slept** the night before (independent variable sleep, S) 
* whether she/he was well **fed** (independent variable, F) 
* how much he or she has been **training** recently (independent variable, T)

You will look at using multiple features in the breakout room
<hr style="border:1px solid grey"> </hr>


## 5.4 Many independent variables - Boston Housing data

What does it contain? You saw it in Tutorials 3 and 4! Let's load it again. For more info - https://scikit-learn.org/stable/datasets/index.html#boston-dataset

### 5.4.1 Load the Boston data into a dictionary a dataframe and X and y numpy arrays

```
# Load the dataset into a dictionary and into a dataframe
boston_dict = datasets.load_boston() # A dictionary object 
boston_df = pd.DataFrame(boston_dict.data, columns=boston_dict.feature_names) # A Pandas dataframe
boston_df['MEDV'] = boston_dict.target # Define the dependent variable as the housing cost 'MEDV'
```

In [ ]:
# Load the dataset into a dictionary and into a dataframe
boston_dict = datasets.load_boston() # A dictionary object 
boston_df = pd.DataFrame(boston_dict.data, columns=boston_dict.feature_names) # A Pandas dataframe
boston_df['MEDV'] = boston_dict.target # Define the dependent variable as the housing cost 'MEDV'

#### Exercise 5
Check that the dataframe looks good by printing out it's content


```
# X will contain the independent variables and y the dependent variable
X, y = datasets.load_boston(return_X_y=True) # To be used when we want to carry out regression
```

In [ ]:
# X will contain the independent variables and y the dependent variable
X, y = datasets.load_boston(return_X_y=True) 

### 5.4.2 Visualizing the housing prices - the dependent variable, or "output label"

The value we aim to predict or evaluate is the price of housing. This is our dependent variable. Respect to the case of the Cheetah, we will look at how it is related to the 13 independent variables, also known as *input features*. These variables were listed a few cells above and inclue i.e. CRIM, TAX etc...

```
# We can make a histogram as seen in Tutorial 4
boston_df['MEDV'].plot(kind='hist', bins=30, color="rebeccapurple", alpha=0.8)
plt.xlim([0,60])
plt.show()

```

In [ ]:
# We can make a histogram as seen in Tutorial 4
boston_df['MEDV'].plot(kind='hist', bins=30, color="rebeccapurple", alpha=0.8)
plt.xlim([0,60])
plt.show()


### 5.4.3 Recall from Tutorial 4 the 13 independent variables or "input features"

You saw that by using pairwise correlation, see e.g. https://en.wikipedia.org/wiki/Pearson_correlation_coefficient you could identify how correlated these variables were

```
fig, ax = plt.subplots(1, 1, figsize = (8,8))

# create a mask to white-out the upper triangle
mask = np.triu(np.ones_like(boston_df.corr(), dtype=bool))

# we'll want a divergent colormap for this so our eye
# is not attracted to the values close to 0
cmap = sns.diverging_palette(230, 20, as_cmap=True)

sns.heatmap(boston_df.corr(), mask=mask, cmap=cmap, ax=ax)

plt.show()
```

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (6,6))

# create a mask to white-out the upper triangle
mask = np.triu(np.ones_like(boston_df.corr(), dtype=bool))

# we'll want a divergent colormap for this so our eye
# is not attracted to the values close to 0
cmap = sns.diverging_palette(230, 20, as_cmap=True)

sns.heatmap(boston_df.corr(), mask=mask, cmap=cmap, ax=ax)

plt.show()

### 5.4.4 Linear regression on the entire Boston dataset 



```
# Create linear regression object - note that we are using all the input features
regr = linear_model.LinearRegression()
regr.fit(X, y)
Y_calc = regr.predict(X)
```



In [ ]:
# Create linear regression object - note that we are using all the input features
regr = linear_model.LinearRegression()
regr.fit(X, y)
Y_calc = regr.predict(X)

Let's see what the coefficients look like ... 

```
print("Fit coefficients: \n", regr.coef_, "\nNumber of coefficients:", len(regr.coef_))
```

In [ ]:
print("Fit coefficients: \n", regr.coef_, "\nNumber of coefficients:", len(regr.coef_))

We have 13 !!! That's because we are regressing respect to all **13 independent variables**!!!

So now, $$y_{\sf calc}= m_1x_1 +\, m_2x_2 \,+ \,m_3x_3 \,+\,... \,+ \,b =\sum_{i=1}^{13}m_i x_i + b\;\;\;\;\; \sf eq. 7$$

```
print("We have 13 slopes / weights:\n\n", regr.coef_)
print("\nAnd one intercept: ", regr.intercept_)
```

In [ ]:
print("We have 13 slopes / weights:\n\n", regr.coef_)
print("\nAnd one intercept: ", regr.intercept_)

```
# This size should match the number of columns in X
if len(X[0]) == len(regr.coef_):
    print("All good! The number of coefficients matches the number of input features.")
else:
    print("Hmm .. something strange is going on.")
```

In [ ]:
# This size should match the number of columns in X
if len(X[0]) == len(regr.coef_):
    print("All good! The number of coefficients matches the number of input features.")
else:
    print("Hmm .. something strange is going on.")

Let's **evaluate the error** by computing the MSE and $R^2$ metrics (see eq. 3 and 6).

```
# The mean squared error
print('Mean squared error: %.2f' % mean_squared_error(y, Y_calc))
print('Coefficient of determination: %.2f' % r2_score(y, Y_calc))
```

In [ ]:
# The mean squared error
print('Mean squared error: %.2f' % mean_squared_error(y, Y_calc))
print('Coefficient of determination: %.2f' % r2_score(y, Y_calc))

We can also look at how well the computed values match the true values graphically by generating a scatterplot.

```
# Scatterplot
sns.scatterplot(x=Y_calc, y=y, color="cornflowerblue", s=50)
plt.title("Linear regression - computed values on entire data set", fontsize=16)
plt.xlabel("y$^{\sf calc}$")
plt.ylabel("y$^{\sf true}$")
plt.show()
```

In [ ]:
# Scatterplot
sns.scatterplot(x=Y_calc,y=y, color="cornflowerblue", s=50)
plt.title("Linear regression - computed values on entire data set", fontsize=16)
plt.xlabel("y$^{\sf calc}$")
plt.ylabel("y$^{\sf true}$")
plt.show()

### 5.4.5 Let's use regression to predict the housing cost

To see whether we can predict, we will carry out our regression only on a part, 70%, of the full data set. This part is called the **training** data. We will then test the trained model to predict the rest of the data, 30% - the **test** data. The function which fits won't see the test data until it has to predict it. 

We start by splitting out data using scikit-learn's <code> train_test_split() </code> function:

<code> X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=9) </code>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=9)

Now we check the size of <code> y_train </code> and <code> y_test </code>, the sum should be the size of y! If this works then we move on and carry out regression but we only use the training data!

```
if len(y_test)+len(y_train) == len(y):
    
    print('All good, ready to C-HACK and regress!\n')
    
    # Carry out linear regression
    print('Running linear regression algorithm on the training set\n')
    regr = linear_model.LinearRegression()
    regr.fit(X_train, y_train)
    print('Fit coefficients and intercept:\n\n', regr.coef_, '\n\n', regr.intercept_ )

    # Predict on the test set
    Y_calc_test = regr.predict(X_test)
```

In [ ]:
if len(y_test)+len(y_train) == len(y):
    
    print('All good, ready to C-HACK and regress!\n')
    
    # Carry out linear regression
    print('Running linear regression algorithm on the training set\n')
    regr = linear_model.LinearRegression()
    regr.fit(X_train, y_train)
    print('Fit coefficients and intercept:\n\n', regr.coef_, '\n\n', regr.intercept_ )

    # Predict on the test set
    Y_calc_test = regr.predict(X_test)

Now we can plot our predicted values to see how accurate we are in predicting. We will generate a scatterplot and computing the MSE and $R^2$ metrics of error.

```
sns.scatterplot(x=Y_calc_test, y=y_test, color="mediumvioletred", s=50)
plt.title("Linear regression - predict test set", fontsize=16)
plt.xlabel("y$^{\sf calc}$")
plt.ylabel("y$^{\sf true}$")
plt.show()

print('Mean squared error: %.2f' % mean_squared_error(y_test, Y_calc_test))
print('Coefficient of determination: %.2f' % r2_score(y_test, Y_calc_test))
```

In [ ]:
sns.scatterplot(x=Y_calc_test, y=y_test, color="mediumvioletred", s=50)

plt.title("Linear regression - predict test set", fontsize=16)
plt.xlabel("y$^{\sf calc}$")
plt.ylabel("y$^{\sf true}$")
plt.show()

print('Mean squared error: %.2f' % mean_squared_error(y_test, Y_calc_test))
print('Coefficient of determination: %.2f' % r2_score(y_test, Y_calc_test))

#### Exercise 6
Play with the size of the train and test set - how does that change the error?

### 5.4.6 Do we need all independent variables? 

We will look at prediction ability as a function of input features in the BREAKOUT ROOM, right after the break!

### 5.4.7 Are there other ways to carry our regression?

Yes! There are many - for example Ridge Regression, LASSO, Random Forests ... you will see one of these in the BREAKOUT ROOM / one example below

```
regr = linear_model.Ridge()
regr.fit(X_train, y_train)
print('Fit coefficients and intercept:\n\n', regr.coef_, '\n\n', regr.intercept_ )

# Predict on the test set
Y_calc_test = regr.predict(X_test)
```

In [ ]:
regr = linear_model.Ridge()
regr.fit(X_train, y_train)
print('Fit coefficients and intercept:\n\n', regr.coef_, '\n\n', regr.intercept_ )

# Predict on the test set
Y_calc_test = regr.predict(X_test)

```
sns.scatterplot(x=Y_calc_test, y=y_test, color="lightseagreen", s=50)
plt.title("Ridge regression - predict test set",fontsize=16)
plt.xlabel("y$^{\sf calc}$")
plt.ylabel("y$^{\sf true}$")
plt.show()

print('Mean squared error: %.2f' % mean_squared_error(y_test, Y_calc_test))
print('Coefficient of determination: %.2f' % r2_score(y_test, Y_calc_test))
```

In [ ]:
sns.scatterplot(x=Y_calc_test, y=y_test, color="lightseagreen", s=50)
plt.title("Ridge regression - predict test set",fontsize=16)
plt.xlabel("y$^{\sf calc}$")
plt.ylabel("y$^{\sf true}$")
plt.show()

print('Mean squared error: %.2f' % mean_squared_error(y_test, Y_calc_test))
print('Coefficient of determination: %.2f' % r2_score(y_test, Y_calc_test))

# Breakout Room


### Problem 1) Number and choice of input features / Bias and variance

Load the Boston dataset and evaluate how the linear regression predictions changes as you change the **number and choice of input features**. The total number of columns in X  is 13 and each column represent a specific input feature. Estimate the MSE and compute the **variance / estimate the bias** for different training dataset slices.
```
print(X_train.shape)
```

In [ ]:
print(X_train.shape)

If you want to use the first 5 features you could proceed as following:

```
X_train_five = X_train[:,0:5]
X_test_five = X_test[:,0:5]
```

In [ ]:
X_train_five = X_train[:,0:5]
X_test_five = X_test[:,0:5]

Check that the new variables have the shape your expect

```
print(X_train_five.shape)
print(X_test_five.shape)
```

In [ ]:
print(X_train_five.shape)
print(X_test_five.shape)

Now you can use these to train your linear regression model and repeat for different numbers or sets of input features! Note that you do not need to change the output feature! It's size is independent from the number of input features, yet recall that its length is the same as the number of values per input feature.

Questions to think about while you work on this problem
- How many input feature variables does one need? Is there a maximum or minimum number? 
- Are the conclusions you make on one dataset general?
- Could one input feature variable be better than the rest?
- What if values are missing for one of the input feature variables - is it still worth using it?
- How do the bias and variance change for each method? Try using different slices of your training dataset and compare the predictions as we did for the Cheetah

### Problem 2) Type of regression algorithm


If you are already a Python expert and want to try something different, try using other types of linear regression methods on the Boston dataset: the LASSO model and the Elastic net model which are described by the 

<code > sklearn.linear_model.ElasticNet() </code> <br>
<code > sklearn.linear_model.Lasso() </code>

scikit-learn functions.

For more detail see [ElasticNet](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet) and [Lasso](  https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso).

Questions to think about while you work on this problem
- How does the error change with each model?
- Which model seems to perform best?
- Does one model do better than the other at determining which input features are more important?
- How about non linear regression / what if the data does not follow a line?
- How do the bias and variance change for each model

<hr style="border:1px solid grey"> </hr>

# References

 * Video of a Cheetah: https://www.youtube.com/watch?v=8-9oFxYFODE

* **Linear Regression**
To find out more see https://en.wikipedia.org/wiki/Simple_linear_regression

* **scikit-learn**
 * Scikit-learn: https://scikit-learn.org/stable/
 * Linear regression in scikit-learn: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
 * Metrics of error: https://scikit-learn.org/stable/modules/model_evaluation.html
  * The Boston dataset: https://scikit-learn.org/stable/datasets/index.html#boston-dataset

* **Pearson correlation**
To find out more see https://en.wikipedia.org/wiki/Pearson_correlation_coefficient

* **Irreducible error, bias and variance**
 * Great Coursera videos here: https://www.coursera.org/lecture/ml-regression/irreducible-error-and-bias-qlMrZ
and here: https://www.coursera.org/lecture/ml-regression/variance-and-the-bias-variance-tradeoff-ZvP40
